**Improve to a higher salary level. A global scale perspective :
Ability dependence, Experience dependence, both or none of these?**

In ML generation,  companies and industries looking for more expertise to help analyze and predict how their products fit the need of customers and markets through out the world. Instead, ML has been applied to multiple fields of industries. As a consequence, the more precise prediction a worker can provide, the higher salary the worker can earn. However, behind an accurate prediction and a higher salary level, there were lots of effort needs to be contributed. Namely, what skills, experience, and learning attitude a worker should prepare determined the salary level. Therefore, inorder to found out what determined the compensation level and difference between districts, in this notebook, I will provide the following steps to analysed and visualised the datasets based on this survey:

1. Feature cleaning
2. Feature engineering
3. Selecting feature
4. Visualised these features


In [ ]:
#dataframe import
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#for statas
from scipy import stats

#visualsing
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib import rc


#Data featuring
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin


In [ ]:
#Useful function to extract columns and count ability

def Extract_columns(data, Select_condition, act):
    columnname = list(data.iloc[1].index)
    
    Qstep1 = []
    for word in columnname:
        re = Select_condition in word
        Qstep1.append(re)
        
    Q = []
    for var in list(np.where(Qstep1)[0]):
        re = columnname[var]
        Q.append(re)
    
    if act == "remove":
        re = data.drop(Q, axis = 1)
    elif act == "Convert":
        data[Q] = data[Q].notnull().astype('int')
    else:
        re = data[Q]

    return(re)


def CountNumberAbility(data, dropvalue, select_condition,act):
    
    x_testQQ = data.drop([dropvalue], axis = 1)
    QQ = Extract_columns(data = x_testQQ , Select_condition = select_condition, act = act)
    
    QQ = QQ.notnull().astype('int')
    
    number_of_ability = QQ.sum(axis=1)
    return(number_of_ability)


#Outlier detection by IQR
def OutlierRemove(data, listoftarget):
    
    tg = data[listoftarget]
    
    Q1 = tg.quantile(0.25)
    Q3 = tg.quantile(0.75)
    IQR = Q3 - Q1
    
    data_out = data[~((tg < (Q1 - 1.5 * IQR)) |(tg > (Q3 + 1.5 * IQR))).any(axis=1)]
    
    return(data_out)

#for modeling 
def rmse(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    return mean_squared_error(y_test, y_pred, squared= False)

**Process of hadling the survey data**
1. Countries were group as differernt district based on their geolocation. 
2. The nomial variables were transform to dummie format to prepared for feature selection. 
3. Those ordinal variables were transform to numeric number. 
3. Adding features based on the counts of prgramming language, database, environment, method of machine learning, method of visulising, and modeling approach that a worker is familiar with as ability.  
4. Remove the outlier from those numeric variable if they present.
5. Dropped those questions that is not correlated with ability or attitude (e.g. questions relates to the company scale of a worker belongs to and gender of a human).

In [ ]:
class FeatureEngineering(BaseEstimator, TransformerMixin):
    def fit(self, X, y= None):
        self.drop_ls = ['Q8',
                        'Q1',
                        'Q2',
                        'Q5',
                        'Q20',
                        'Q21',
                        'Q25',
                        'Q37_Part_1',
                        'Q37_Part_2',
                        'Q37_Part_3',
                        'Q37_Part_4',
                        'Q37_Part_5',
                        'Q37_Part_6',
                        'Q37_Part_7',
                        'Q37_Part_8',
                        'Q37_Part_9',
                        'Q37_Part_10',
                        'Q37_Part_11',
                        'Q39_Part_1',
                        'Q39_Part_2',
                        'Q39_Part_3',
                        'Q39_Part_4',
                        'Q39_Part_5',
                        'Q39_Part_6',
                        'Q39_Part_7',
                        'Q39_Part_8',
                        'Q39_Part_9',
                        'Q39_Part_10',
                        'Q39_Part_11',
                        'Time from Start to Finish (seconds)'
                      ]
    
    def transform(self, X, y= None): 
        
        #Removed the column include "OTHER" to ensure the suffecient information that gain from this survey
        X = Extract_columns(X, "OTHER", "remove")
        
        #Reclassifying featureing
        X["Q24"].replace({"$0-999": 1, 
                       "1,000-1,999": 2,
                       "2,000-2,999": 2, 
                       "3,000-3,999": 2, 
                       "4,000-4,999": 2, 
                       "5,000-7,499": 3, 
                       "7,500-9,999": 3,
                       "10,000-14,999":4, 
                       "15,000-19,999":4, 
                       "20,000-24,999":5,
                       "25,000-29,999":5, 
                       "30,000-39,999":6,
                       "40,000-49,999":7,
                       "50,000-59,999":8,
                       "60,000-69,999":9,
                       "70,000-79,999":10,
                       "80,000-89,999":11,
                       "90,000-99,999":12,
                       "100,000-124,999":13,
                       "125,000-149,999":14,
                       "150,000-199,999":15,
                       "200,000-249,999":16,
                       "250,000-299,999":17,
                       "300,000-500,000":18,
                       "> $500,000":19,}, inplace=True)
        
        X["Q3"].replace({"India": "South_Asia", 
                       "United States of America": "North_America",
                       "Other": "Other", 
                       "Brazil": "South_America", 
                       "Japan": "East_Asia", 
                       "Russia": "North_Asia", 
                       "United Kingdom of Great Britain and Northern Ireland": "West_Europe",
                       "Germany":"Central_Europe", 
                       "Nigeria":"Africa", 
                       "Spain":"South_Europe",
                       "Canada":"North_America", 
                       "France":"West_Europe",
                       "Italy":"South_Europe",
                       "China":"East_Asia",
                       "Turkey":"West_Asia",
                       "Australia":"Australia",
                       "Mexico":"South_America",
                       "Indonesia":"Southeast_Asia",
                       "Taiwan":"East_Asia",
                       "Pakistan":"South_Asia",
                       "Colombia":"South_America",
                       "Ukraine":"East_Europe",
                       "Netherlands":"West_Europe",
                       "Poland":"Central_Europe",
                       "Egypt":"Africa",
                       "Singapore":"Southeast_Asia",
                       "Portugal":"South_America",
                       "South Korea":"East_Asia",
                       "Argentina":"South_America",
                       "Viet Nam":"Southeast_Asia",
                       "South Africa":"Africa",
                       "Iran, Islamic Republic of...":"West_Asia",
                       "Kenya":"Africa",
                       "Thailand":"Southeast_Asia",
                       "Greece":"South_Europe",
                       "Israel":"West_Asia",
                       "Peru":"South_America",
                       "Chile":"South_America",
                       "Morocco":"Africa",
                       "Sweden":"North_Europe",
                       "Malaysia":"Southeast_Asia",
                       "Philippines":"Southeast_Asia",
                       "Tunisia":"Africa",
                       "Switzerland":"Central_Europe",
                       "Saudi Arabia":"West_Asia",
                       "Bangladesh":"South_Asia",
                       "United Arab Emirates":"West_Asia",
                       "Republic of Korea":"East_Asia",
                       "Sri Lanka":"South_Asia",
                       "Romania":"South_Europe",
                       "Ghana":"Africa",
                       "Belarus":"East_Europe", 
                       "Belgium":"West_Europe", 
                       "Ireland":"West_Europe", 
                       "Nepal":"South_Asia"}, inplace=True)
        
       
        
       
        #Adding new feature: These feature were created based on the number of skill or abililty that a person had learn or famamilar with.
        #I dopped the column "None" to ensure it is not one of the ability they learnt.
        
        number_of_code= CountNumberAbility(data = X, dropvalue = "Q7_Part_11", select_condition = "Q7", act = "non")
        number_of_env= CountNumberAbility(data = X, dropvalue = "Q9_Part_10", select_condition = "Q9", act = "non") 
        number_of_notebook= CountNumberAbility(data = X, dropvalue = "Q10_Part_13", select_condition = "Q10", act = "non")   
        number_of_hardware = CountNumberAbility(data = X, dropvalue = "Q12_Part_2", select_condition = "Q12", act = "non")   
        number_of_visualised = CountNumberAbility(data = X, dropvalue = "Q14_Part_11", select_condition = "Q14", act = "non")   
        number_of_ML = CountNumberAbility(data = X, dropvalue = "Q16_Part_15", select_condition = "Q16", act = "non")   
        number_of_AL = CountNumberAbility(data = X, dropvalue = "Q17_Part_11", select_condition = "Q17", act = "non")   
        number_of_vision = CountNumberAbility(data = X, dropvalue = "Q18_Part_6", select_condition = "Q18", act = "non")   
        number_of_nlp = CountNumberAbility(data = X, dropvalue = "Q19_Part_5", select_condition = "Q19", act = "non")   
        number_of_useage = CountNumberAbility(data = X, dropvalue = "Q23_Part_7", select_condition = "Q23", act = "non")
        number_of_cloud = CountNumberAbility(data = X, dropvalue = "Q26_A_Part_10", select_condition = "Q26_A", act = "non")
        number_of_service = CountNumberAbility(data = X, dropvalue = "Q27_A_Part_10", select_condition = "Q27_A", act = "non")
        number_of_platform = CountNumberAbility(data = X, dropvalue = "Q28_A_Part_10", select_condition = "Q28_A", act = "non")
        number_of_database = CountNumberAbility(data = X, dropvalue = "Q29_A_Part_16", select_condition = "Q29_A", act = "non")
        number_of_BI = CountNumberAbility(data = X, dropvalue = "Q31_A_Part_14", select_condition = "Q31_A", act = "non")
        number_of_AutoML = CountNumberAbility(data = X, dropvalue = "Q33_A_Part_7", select_condition = "Q33_A", act = "non")
        number_of_AutoMLskill = CountNumberAbility(data = X, dropvalue = "Q34_A_Part_11", select_condition = "Q34_A", act = "non")
        number_of_MLtool = CountNumberAbility(data = X, dropvalue = "Q35_A_Part_10", select_condition = "Q35_A", act = "non")
        number_of_shareing = CountNumberAbility(data = X, dropvalue = "Q36_Part_9", select_condition = "Q36", act = "non")
        
        
        X['Environment_ability'] = number_of_env
        X['Code_ability'] = number_of_code
        X['Notebook_ability'] = number_of_notebook
        X['Hardware_ability'] = number_of_hardware
        X['Visualised_ability'] = number_of_visualised
        X['ML_ability'] = number_of_ML
        X['AL_ability'] = number_of_AL
        X['Vision_ability'] = number_of_vision
        X['nlp_ability'] = number_of_nlp
        X['Numberof_applyfield'] = number_of_useage
        X['Cloud_ability'] = number_of_cloud
        X['Service_ability'] = number_of_service
        X['platform_ability'] = number_of_platform
        X['database_ability'] = number_of_database
        X['BI_ability'] = number_of_BI             
        X['AutoML_ability'] = number_of_AutoML
        X['AutoMLskill_ability'] = number_of_AutoMLskill
        X['MLtool_ability'] = number_of_MLtool
        X['Sharing_of_heart'] = number_of_shareing

        
        
        #Replace the threshold variable to numeric value
        X['Q4'].replace(list(X['Q4'].value_counts().index),[4,3,5,6,2,0,1], inplace = True)
        X['Q6'].replace(list(X['Q6'].value_counts().index),[3,2,4,5,1,6,0], inplace = True)
        X['Q11'].replace(list(X['Q11'].value_counts().index),[1,2,3,4,0], inplace = True)
        X['Q13'].replace(list(X['Q13'].value_counts().index), [2,3,4,0,1], inplace = True)
        X['Q15'].replace(list(X['Q15'].value_counts().index), [1,2,3,0,4,6,5,7,8], inplace = True)
        X['Q22'].replace(list(X['Q22'].value_counts().index), [3,0,5,4,1,2], inplace = True)
        
        
        
        #Replace the skill ability to 1 and 0
        Extract_columns(data = X, Select_condition = "Q7" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q9" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q10" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q12" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q14" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q19" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q16" , act = "Convert")  
        Extract_columns(data = X, Select_condition = "Q17" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q18" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q23" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q26" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q27" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q28" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q29" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q31" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q33" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q35" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q34" , act = "Convert")
        Extract_columns(data = X, Select_condition = "Q36" , act = "Convert")
      
        #Outlier removing which is based on the scatter plot between compensation level and the below feature
        X = X.loc[X['Sharing_of_heart'] < 6]
        X = X.loc[X['MLtool_ability'] < 6]
        X = X.loc[X['platform_ability'] < 8]
        X = X.loc[X['AutoMLskill_ability'] < 7]
        X = X.loc[X['AutoML_ability'] < 7]
        X = X.loc[X['BI_ability'] < 7]
        X = X.loc[X['database_ability'] < 11]
        X = X.loc[X['Service_ability'] < 9]
        X = X.loc[X['Cloud_ability'] < 8]
        X = X.loc[X['ML_ability'] < 13]
        X = X.loc[X['Visualised_ability'] < 10]
        X = X.loc[X['Code_ability'] < 10]
        X = X.loc[X['Environment_ability'] < 9]   
        
        #Drop the NA rows of Q11, Q13, Q15, Q17 but not filling it is because they are indepandant variable.
        #Therfore it is not feasible insert it with the most ovserved feature or average feature
        
        X = X[X['Q11'].notna()]
        X = X[X['Q25'].notna()]
        
        
        for feature in self.drop_ls:
            X.drop([feature], axis=1, inplace=True)
        return X
    
    def fit_transform(self, X, y=None):
        
        self.fit(X, y)
        return self.transform(X, y)

In [ ]:
#Transform these variable to dummy variable
def getrans(X):
    X = pd.get_dummies(X,columns=["Q30"])
    X = pd.get_dummies(X,columns=["Q32"])
    X = pd.get_dummies(X,columns=["Q38"])
    X = Extract_columns(X, "Other", "remove")
    return(X)

In [ ]:
#Data import
df_test = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')
df_test= df_test.drop([0], axis = 0)
df_test.head()

In [ ]:
#To begin with this project, I remove the nan row based on "Q24 What is your current yearly compensation (approximate $USD)"

x_test = df_test.copy()
x_test = Extract_columns(data = x_test,Select_condition = "B", act = "remove")
x_test = x_test[x_test['Q24'].notna()]
x_test.shape

In [ ]:
#Dealing with features by the feature engineering above
ft = FeatureEngineering()
ft.fit(x_test)
x_test = ft.transform(x_test)

x_test = getrans(x_test)
x_test

In [ ]:
#To check whether two variable in the datasets correlating with each other         
Numeric_variable = ['Environment_ability','Code_ability','Notebook_ability','Hardware_ability',
                    'Visualised_ability','ML_ability','AL_ability','Vision_ability','nlp_ability',
                    'Numberof_applyfield','Cloud_ability','Service_ability','platform_ability','database_ability',
                    'BI_ability','AutoML_ability','AutoMLskill_ability','MLtool_ability','Sharing_of_heart','Q4','Q6','Q11','Q22','Q24'
                   ]


Numeric = x_test[Numeric_variable]
corr = Numeric.corr()

#The true here stands for keeping the variable and no auto correlation appear between features.
m = ~(corr.mask(np.eye(len(corr), dtype=bool)).abs() > 0.85).any()
m

In [ ]:
#Plot the correlation heat map
plt.subplots(figsize=(15,12))
sns.heatmap(data = corr)

In [ ]:
#Checking outlier of compensation level by IQR
x_test.shape

In [ ]:
#Checking outlier of compensation level by IQR

listoftarget = ['Q24']
out = OutlierRemove(data = x_test,listoftarget = listoftarget)
out.shape

In [ ]:
Countrygroup = (x_test.groupby(['Q3'])['Q24']
          .value_counts(normalize=True)
          .rename('percentage')
          .mul(100)
          .reset_index())
Countrygroup = Countrygroup.loc[Countrygroup['Q3'] != "Other"]
g = sns.FacetGrid(Countrygroup, col="Q3", col_wrap= 3)
g.map(sns.barplot,"Q24", "percentage")
g.set(xlabel="Compensation Levels")

**Unblance compensation level among districts** 
* The histogram shows differernt salary structure between districts. In Africa and South Asia, those people who earn less than 999 dolars per year is more than 50%. The structure is more even in Europe and North America.

In [ ]:
#Modeling
X = x_test.drop(['Q24','Q3'],axis = 1)
y = x_test['Q24']

**Model selection to found out which criteria determined the compensation level**
* By applying SelectKBest based on F-test and ExtraTreesClassifier, I found that the most three influencial factors are the expreience of ML you learnt and apply and the experience of coding. (i.g. Q6, Q15, Q22) This result infered that instead of chosing what type of program language to learn or how many types of programing language to learn, people who is looking for a higher compensation level should focus on more projects relate to ML to gain experience.  

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from numpy import set_printoptions

test = SelectKBest(score_func=f_classif, k=4)
fit = test.fit(X, y)
# summarize scores
set_printoptions(precision=3)


impo = list(fit.scores_)
col = list(X.columns) 
d = {'column':col,'Importance':impo}

Importanceresult = pd.DataFrame(d).sort_values(['Importance'], ascending=False)
Importanceresult = Importanceresult.head(3)
Importanceresult['Full Question name'] = ['For how many years have you been writing code and/or programming?','For how many years have you used machine learning methods?','Does your current employer incorporate machine learning methods into their business?']

Importanceresult

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


model = ExtraTreesClassifier(n_estimators=10)
model.fit(X, y)

impo = list(model.feature_importances_)
col = list(X.columns) 
d = {'column':col,'Importance':impo}
Importanceresult = pd.DataFrame(d).sort_values(['Importance'],ascending=False)

#Setup a threshold
thres = Importanceresult['Importance'].mean()*3

#Extract the column
Selectimpo = Importanceresult.loc[Importanceresult['Importance'] > thres]



Selectimpo = Selectimpo.head(3)
Selectimpo['Full Question name'] = ['For how many years have you been writing code and/or programming?','For how many years have you used machine learning methods?','Does your current employer incorporate machine learning methods into their business?']

Selectimpo


In [ ]:
#Visualised Q6 with compensation level
#sns.barplot(data = x_test, x = 'Q6' , y  = 'Q24')
color_palette = ['#620800','#ff1500','#ff5500','#ff9500', '#ffd500','#e9ff00']
dc = {'Q6':list(x_test.groupby(['Q6'])['Q24'].mean().index),
      'Value':list(x_test.groupby(['Q6'])['Q24'].mean().values),
      'Std':list(x_test.groupby(['Q6'])['Q24'].std().values)}
Q6 = pd.DataFrame(dc)

#list(x_test.groupby(['Q6'])['Q24'].mean().index)
positions = (1, 2, 3, 4, 5 ,6)
labels = ("< 1 years", "1-2 years", "3-5 years", "5-10 years", "10-20 years", "20+ years")
plt.rcParams["figure.figsize"] = (10, 6)
barWidth = 0.85
plt.bar(Q6['Q6'], Q6['Value'], yerr=Q6['Std'],color= color_palette[1], edgecolor='white', width=barWidth, align='center', alpha=0.5, ecolor='black', capsize=10)
plt.xticks(positions, labels)
plt.xlabel("Question 6", fontsize = 16)
plt.ylabel("Average_Compensaionlevel", fontsize = 16)
plt.title("For how many years have you been writing code and/or programming?", fontsize = 16)

From bar plot result of question 6, we can tell that the more coding experience, the higher compensation level a person will earn. Namely, concentrate on few particular programming language is enough to solve differernt problem on projects. This result is disagree with what I expected which is the more programming language a person known the more salary he/she will earn. The reason might due to it is more important how you solve the problem by your familiar language instead of knowing lots of programming languages and keep the problem unsolved.

In [ ]:
#Visualised Q6 with compensation level
dc = {'Q15':list(x_test.groupby(['Q15'])['Q24'].mean().index),
      'Value':list(x_test.groupby(['Q15'])['Q24'].mean().values),
      'Std':list(x_test.groupby(['Q15'])['Q24'].std().values)}
Q15 = pd.DataFrame(dc)
plt.rcParams["figure.figsize"] = (10, 6)
plt.bar(Q15['Q15'], Q15['Value'], yerr=Q15['Std'],color= color_palette[1], edgecolor='white', width=barWidth, align='center', alpha=0.5, ecolor='black', capsize=10)
plt.rcParams["figure.figsize"] = (10, 6)
positions = (0, 1, 2, 3, 4, 5 ,6 ,7 ,8 )
labels = ("I do not use","< 1 years", "1-2 years", "2-3 years", "3-4 years","4-5 years","5-10 years", "10-20 years", "20+ years")

plt.xticks(positions, labels, rotation=45)
plt.xlabel("Question 15", fontsize = 16)
plt.ylabel("Average_Compensaionlevel", fontsize = 16)
plt.title("For how many years have you used machine learning methods?",  fontsize = 16)

From bar plot result of question 15, it goes similar as the result of question 6. Here, the result showed that the more time spending on machine learning the more money you earn. On the other words, the salary level is depended on how fast you get familiar with the job. Besides, those people who work on machine learning for only 1 - 3 year might not learn enough experience from their job. This result could give us a hint that if you want to imporve yourself to a better compensation level it might not be immediate.

In [ ]:
#Visualised Q6 with compensation level
#sns.barplot(data = x_test, x = 'Q22' , y  = 'Q24')

dc = {'Q22':list(x_test.groupby(['Q22'])['Q24'].mean().index),
      'Value':list(x_test.groupby(['Q22'])['Q24'].mean().values),
      'Std':list(x_test.groupby(['Q22'])['Q24'].std().values)}
Q22 = pd.DataFrame(dc)

plt.rcParams["figure.figsize"] = (10, 6)
plt.bar(Q22['Q22'], Q22['Value'], yerr=Q22['Std'],color= color_palette[1], edgecolor='white', width=barWidth, align='center', alpha=0.5, ecolor='black', capsize=10)
positions = (0,1, 2, 3, 4, 5 )
labels = ("do not know", "not use", "generating insights", "exploring", "recently started", "well established")

plt.xticks(positions, labels, rotation=45)
plt.xlabel("Question 22", fontsize = 16)
plt.ylabel("Average_Compensaionlevel", fontsize = 16)
plt.title("Does your current employer incorporate machine learning methods into their business?",  fontsize = 16)

From bar plot result of question 22, the highest average compensation level is fall into the "employer have well established ML methods" group. Although this result is not directly link to what ability or what atitude that need to be prepared, it still tells us, again, establishing ML methods is a trend for company. Therfore, link to the result showed in question 15, the people who are more familiar with Machine Learning, the higher compensation level they earn.  

**Comparison the three most important factors by differernt district**
* From the bar plots below, it is easily to fonud that most of the developing district is still lacking of the experience of coding. Not to mention a well developing maching learning skills. Most of the number of the people who had less written code and used machine learning approach is dominant by Africa. This also reflect the compensation level of their district. While the case is totaly oppsite in those well developing district such as North America and North Europe. These result not only reflect the compensation level but also the education structure, since the ability of coding and machine learning is based on well developed education system. However, this system could not be found since people in those district might confronting problem of survive. Besides, from these results we might able to discover the gap of technology between those developing districts and well developed districts.

In [ ]:
x_test = x_test.loc[x_test['Q3'] != 'Other']
Countrygroup = (x_test.groupby(['Q3'])['Q6']
          .value_counts(normalize=True)
          .rename('percentage')
          .mul(100)
          .reset_index())
Countrygroup = Countrygroup.sort_values(['Q3'], ascending=False)

plt.rcParams["figure.figsize"] = (10, 6)

#district = list(Countrygroup['Q3'].value_counts().index)
Question_level = list(Countrygroup['Q6'].value_counts().index)

#Question_level



legend = ['< 1 years', '1 - 2 years']



first = Countrygroup.loc[Countrygroup['Q6'] == Question_level[5]].sort_values(['Q3'], ascending=False)
second = Countrygroup.loc[Countrygroup['Q6'] == Question_level[4]].sort_values(['Q3'], ascending=False)

first['sum'] = first['percentage'].values + second['percentage'].values
second['sum'] = first['percentage'].values + second['percentage'].values

first = first.sort_values(['sum'], ascending=False)#['percentage']
second = second.sort_values(['sum'], ascending=False)['percentage']


plt.bar(first['Q3'], first['percentage'],color= color_palette[1], edgecolor='white', width=barWidth, alpha = 0.6)
plt.bar(first['Q3'], second, bottom = first['percentage'],color= color_palette[3], edgecolor='white', width=barWidth, alpha = 0.6)


plt.xlabel("District", fontsize = 16)
plt.ylabel("Percentage", fontsize = 16)

plt.legend(labels = legend, bbox_to_anchor=(1.02, 1), loc='upper left', fontsize = 14)
plt.xticks(rotation=90,  fontsize = 14)
plt.yticks(fontsize = 14)

plt.title('For how many years have you been writing code and/or programming?', fontsize = 16)

In [ ]:

first = Countrygroup.loc[Countrygroup['Q6'] == Question_level[0]].sort_values(['Q3'], ascending=False)
second = Countrygroup.loc[Countrygroup['Q6'] == Question_level[1]].sort_values(['Q3'], ascending=False)

first['sum'] = first['percentage'].values + second['percentage'].values
second['sum'] = first['percentage'].values + second['percentage'].values

first = first.sort_values(['sum'], ascending=True)#['percentage']
second = second.sort_values(['sum'], ascending=True)['percentage']

legend = ['20+ years','10-20 years']


plt.bar(first['Q3'], first['percentage'],color= color_palette[1], edgecolor='white', width=barWidth, alpha = 0.6)
plt.bar(first['Q3'], second, bottom = first['percentage'],color= color_palette[3], edgecolor='white', width=barWidth, alpha = 0.6)


plt.xlabel("District", fontsize = 16)
plt.ylabel("Percentage", fontsize = 16)

plt.legend(labels = legend, bbox_to_anchor=(1.02, 1), loc='upper left', fontsize = 14)
plt.xticks(rotation=90,  fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('For how many years have you been writing code and/or programming?', fontsize = 16)

In [ ]:
x_test = x_test.loc[x_test['Q3'] != 'Other']
Countrygroup = (x_test.groupby(['Q3'])['Q15']
          .value_counts(normalize=True)
          .rename('percentage')
          .mul(100)
          .reset_index())
Countrygroup = Countrygroup.sort_values(['Q3'], ascending=False)

plt.rcParams["figure.figsize"] = (10, 6)

#district = list(Countrygroup['Q3'].value_counts().index)
Question_level = list(Countrygroup['Q15'].value_counts().index)
# color_palette = ['#620800','#ff1500','#ff5500','#ff9500', '#ffd500','#e9ff00']
Question_level = sorted(Question_level)
Question_level

# barWidth = 0.85
legend = ['I do not use machine learning methods','Under 1 year', '1-2 years']



first = Countrygroup.loc[Countrygroup['Q15'] == Question_level[0]].sort_values(['Q3'], ascending=False)
second = Countrygroup.loc[Countrygroup['Q15'] == Question_level[1]].sort_values(['Q3'], ascending=False)
third = Countrygroup.loc[Countrygroup['Q15'] == Question_level[2]].sort_values(['Q3'], ascending=False)

first['sum'] = first['percentage'].values + second['percentage'].values
second['sum'] = first['percentage'].values + second['percentage'].values
third['sum'] = second['percentage'].values + third['percentage'].values


first = first.sort_values(['sum'], ascending=True)
second = second.sort_values(['sum'], ascending=True)['percentage']
third = third.sort_values(['sum'], ascending=True)['percentage']


plt.bar(first['Q3'], first['percentage'],color= color_palette[1], edgecolor='white', width=barWidth, alpha = 0.6)
plt.bar(first['Q3'], second, bottom = first['percentage'],color= color_palette[2], edgecolor='white', width=barWidth, alpha = 0.6)
plt.bar(first['Q3'], third, bottom=[i+j for i,j in zip(first['percentage'], second)],color= color_palette[3], edgecolor='white', width=barWidth, alpha = 0.6)


plt.xlabel("District", fontsize = 16)
plt.ylabel("Percentage", fontsize = 16)

plt.legend(labels = legend, bbox_to_anchor=(1.02, 1), loc='upper left', fontsize = 14)
plt.xticks(rotation=90,  fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('For how many years have you used machine learning methods', fontsize = 16)

In [ ]:
first = Countrygroup.loc[Countrygroup['Q15'] == Question_level[7]].sort_values(['Q3'], ascending=False)
second = Countrygroup.loc[Countrygroup['Q15'] == Question_level[6]].sort_values(['Q3'], ascending=False)

first['sum'] = first['percentage'].values + second['percentage'].values
second['sum'] = first['percentage'].values + second['percentage'].values

first = first.sort_values(['sum'], ascending=True)#['percentage']
second = second.sort_values(['sum'], ascending=True)['percentage']

legend = ['20+ years','10-20 years']


plt.bar(first['Q3'], first['percentage'],color= color_palette[1], edgecolor='white', width=barWidth, alpha = 0.6)
plt.bar(first['Q3'], second, bottom = first['percentage'],color= color_palette[3], edgecolor='white', width=barWidth, alpha = 0.6)


plt.xlabel("District", fontsize = 16)
plt.ylabel("Percentage", fontsize = 16)

plt.legend(labels = legend, bbox_to_anchor=(1.02, 1), loc='upper left', fontsize = 14)
plt.xticks(rotation=90,  fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('For how many years have you used machine learning methods', fontsize = 16)

In [ ]:
Countrygroup = (x_test.groupby(['Q3'])['Q22']
          .value_counts(normalize=True)
          .rename('percentage')
          .mul(100)
          .reset_index())
Countrygroup = Countrygroup.sort_values(['Q3'], ascending=False)

plt.rcParams["figure.figsize"] = (10, 6)

#district = list(Countrygroup['Q3'].value_counts().index)
Question_level = list(Countrygroup['Q22'].value_counts().index)
color_palette = ['#620800','#ff1500','#ff5500','#ff9500', '#ffd500','#e9ff00']
Question_level = sorted(Question_level)
Question_level

barWidth = 0.85
legend = [' I do not know','No (we do not use ML methods)']



first = Countrygroup.loc[Countrygroup['Q22'] == Question_level[0]].sort_values(['Q3'], ascending=False)
second = Countrygroup.loc[Countrygroup['Q22'] == Question_level[1]].sort_values(['Q3'], ascending=False)

first['sum'] = first['percentage'].values + second['percentage'].values
second['sum'] = first['percentage'].values + second['percentage'].values


first = first.sort_values(['sum'], ascending=False)
second = second.sort_values(['sum'], ascending=False)['percentage']


plt.bar(first['Q3'], first['percentage'],color= color_palette[1], edgecolor='white', width=barWidth, alpha = 0.6)
plt.bar(first['Q3'], second, bottom = first['percentage'],color= color_palette[3], edgecolor='white', width=barWidth, alpha = 0.6)


plt.xlabel("District", fontsize = 16)
plt.ylabel("Percentage", fontsize = 16)

plt.legend(labels = legend, bbox_to_anchor=(1.02, 1), loc='upper left', fontsize = 14)
plt.xticks(rotation=90,  fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('Does your current employer incorporate machine learning methods into their business?', fontsize = 16)

In [ ]:
Countrygroup = (x_test.groupby(['Q3'])['Q22']
          .value_counts(normalize=True)
          .rename('percentage')
          .mul(100)
          .reset_index())
Countrygroup = Countrygroup.sort_values(['Q3'], ascending=False)

plt.rcParams["figure.figsize"] = (10, 6)

#district = list(Countrygroup['Q3'].value_counts().index)
Question_level = list(Countrygroup['Q22'].value_counts().index)
color_palette = ['#620800','#ff1500','#ff5500','#ff9500', '#ffd500','#e9ff00']
Question_level = sorted(Question_level)
Question_level

barWidth = 0.85
legend = [' We have well established ML methods']

first = Countrygroup.loc[Countrygroup['Q22'] == Question_level[5]].sort_values(['Q3'], ascending=False)
first['sum'] = first['percentage'].values
first = first.sort_values(['sum'], ascending=True)

plt.bar(first['Q3'], first['percentage'],color= color_palette[3], edgecolor='white', width=barWidth, alpha = 0.6)


plt.xlabel("District", fontsize = 16)
plt.ylabel("Percentage", fontsize = 16)

plt.legend(labels = legend, bbox_to_anchor=(1.02, 1), loc='upper left', fontsize = 14)
plt.xticks(rotation=90,  fontsize = 14)
plt.yticks(fontsize = 14)
plt.title('Does your current employer incorporate machine learning methods into their business?', fontsize = 16)

**Conclusion**
* In summary, this assay provides a clear answer of what brings a higher compensation level and why some of the districts include a lower compensation level structure. In shorts, earning more money by enhacing our expreience on coding and machine learning is necessary. 